In [1]:
import glob
import geopandas as gpd
import pandas as pd
import numpy as np
from functools import reduce
from scipy.stats import linregress
from plotly import graph_objects as go
from plotly.subplots import make_subplots

In [2]:
gdf = gpd.read_file('/Users/sbhadra/ag-lapse/data/counties.geojson')
csvs = glob.glob('/Users/sbhadra/ag-lapse/data/ag-data/*.csv')
data = []
for csv in csvs:
    df = pd.read_csv(csv, thousands=',', parse_dates=['Year'])
    df = df[['Year', 'State ANSI', 'County ANSI', 'Commodity', 'Data Item', 'Value']]
    df = df.dropna()
    df['State ANSI'] = df['State ANSI'].astype('int').astype('str')
    df['County ANSI'] = df['County ANSI'].astype('int').astype('str')
    df['Id'] = df['State ANSI']+'-'+ df['County ANSI']
    data.append(df)
data = pd.concat(data)
data = data.reset_index(drop=True)
data['Item'] = 0
for i in range(data.shape[0]):
    data.loc[i, 'Item'] = data.loc[i, 'Data Item'].split(' - ')[1]
data = data.pivot(index=['Year', 'Commodity', 'Id'],
                  columns='Item',
                  values='Value')
data = data.reset_index()
data.columns = ['Year', 'Commodity', 'Id', 
               'Harvested', 'Planted', 'Yield']

data = data.merge(right=gdf[['County', 'State', 'Id']],
                  how='left', left_on='Id', right_on='Id')
               
data.to_csv('/Users/sbhadra/ag-lapse/data/ag-data2.csv')

In [3]:
data = pd.read_csv('/Users/sbhadra/ag-lapse/data/ag-data2.csv',
                   index_col=0,
                   parse_dates=['Year'])

In [4]:
data

,Year,Commodity,Id,Harvested,Planted,Yield,County,State
0,1910-01-01,CORN,31-1,87000.0,NaN,31.6,Adams,NE
1,1910-01-01,CORN,31-101,25490.0,NaN,19.9,Keith,NE
2,1910-01-01,CORN,31-103,35730.0,NaN,16.0,Keya Paha,NE
3,1910-01-01,CORN,31-105,7850.0,NaN,9.6,Kimball,NE
4,1910-01-01,CORN,31-107,177990.0,NaN,32.6,Knox,NE
...,...,...,...,...,...,...,...,...
476696,2021-01-01,SOYBEANS,55-91,18000.0,18300.0,55.6,Pepin,WI
476697,2021-01-01,SOYBEANS,55-93,45300.0,46000.0,55.8,Pierce,WI
476698,2021-01-01,SOYBEANS,55-95,36400.0,36800.0,51.4,Polk,WI
476699,2021-01-01,SOYBEANS,55-97,17500.0,17700.0,51.5,Portage,WI


In [3]:
def get_slope(series):
    series = series.sort_index()
    x = np.arange(1, series.shape[0]+1)
    return linregress(x, series.values)[0]

In [4]:
def get_trend(df, start, end, crop, trait):
    start = pd.to_datetime(str(start))
    end = pd.to_datetime(str(end))
    df = df[df['Commodity']==crop]
    df = df[(df['Year']>=start) & (df['Year']<=end)]
    df = df.set_index('Year')
    df = pd.pivot_table(df,
                        values=trait,
                        index=['Id'],
                        aggfunc=get_slope)
    df = df.reset_index()
    return df

In [5]:
start = 2002
end = 2018
crop = 'BARLEY'
trait = 'Harvested'

In [6]:
start = pd.to_datetime(str(start))
end = pd.to_datetime(str(end))
df = data[data['Commodity']==crop]

In [7]:
df = df[(df['Year']>=start) & (df['Year']<=end)]
df = df.set_index('Year')

In [8]:
df

,Commodity,Id,Harvested,Planted,Yield
Year,,,,,
2002-01-01,BARLEY,10-1,12200.0,13300.0,85.0
2002-01-01,BARLEY,10-3,900.0,1000.0,89.0
2002-01-01,BARLEY,10-5,9900.0,10700.0,79.9
2002-01-01,BARLEY,16-1,2700.0,2800.0,99.3
2002-01-01,BARLEY,16-11,22500.0,23300.0,92.8
...,...,...,...,...,...
2018-01-01,BARLEY,6-113,2000.0,5000.0,97.5
2018-01-01,BARLEY,6-79,7800.0,11100.0,57.7
2018-01-01,BARLEY,8-105,15000.0,15300.0,165.1


In [9]:
for idx in np.unique(df['Id']):
    values = df[df['Id']==idx][trait]
    break

In [23]:
fig = make_subplots()

fig.add_trace(
    go.Scatter(
        x=values.index,
        y=values.values,
        name="Median Sale Price ($)",
        mode='lines',
        line = dict(color='firebrick', width=3)
    )
)

fig.update_layout(
    title_text="Housing Market Trends: Atlanta, GA",
    title_font_family="Arial",
    title_font_size = 20,
    title_font_color="darkblue",
    title_x=0.5,
    yaxis_title=trait)

fig.update_xaxes(rangeslider_visible=True)

fig.show()

fig.write_html('/Users/sbhadra/ag-lapse/data/figs/fig1.html')

In [ ]:
    secondary_y=True)
 

In [ ]:
df = pd.pivot_table(df,
                    values=trait,
                    index=['Id'],
                    aggfunc=get_slope)
df = df.reset_index()

In [ ]:
df

In [ ]:
d = get_trend(data, 2001, 2006, 'CORN', 'Yield')

In [ ]:
d.head()

In [ ]:
gdf = gpd.read_file('/Users/sbhadra/ag-lapse/data/cb_2018_us_county_20m.shp')

In [ ]:
gdf['STATEFP'] = gdf['STATEFP'].astype('int').astype('str')
gdf['COUNTYFP'] = gdf['COUNTYFP'].astype('int').astype('str')

In [ ]:
gdf.head()

In [ ]:
gdf['Id'] = gdf['STATEFP']+'-'+ gdf['COUNTYFP']

In [ ]:
gdf.head()

In [ ]:
gdf.to_file('/Users/sbhadra/ag-lapse/data/shape/county.geojson', driver='GeoJSON')

In [ ]:
np.linspace(0, 1, 20)